# LSTM介绍

循环神经网络的变种LSTM模型也有很多种结构，从常规的lstm，到各类复杂结构。

## vanilla lstm

常规的lstm模型。

lstm模型的相关基础知识可以参考：[Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)。

这里简单总结下。首先RNN的结构如图所示：

![](img/RNN-unrolled.png)

RNN中A就是一个激活函数tanh。两个A之间的箭头意思是把第一个A输出的h传递给第二个A。也就是说，第二个A的输入是concatenate了上一个时段的输出和本时段的输入。

那么LSTM就是将A进一步优化，引入了Cell State，通过几个gate来控制节点 来决定记忆哪些，忘记哪些，其中一个四门的形式的RNN就是LSTM。四门如下：

![](img/Picture10.png)

橙色是forget gate layer。

黄色包括两小部分，是为了确定要记忆什么，第一小部分是input gate layer；第二小部分是初始记忆单元作为初始cell state。

绿色是更新cell state，结合遗忘后剩下的历史cell state和刚黄色部分决定本次记忆的cell state 组成新的cell state

蓝色就是最后利用新生成的cell state，结合上个时段的h，生成新的h。

所以和RNN不同，LSTM不同时间段之间的A之间的联系包括h和cell state两种。

github上可以查到各种版本的LSTM，在使用它们之前，有必要了解下pytorch本身自带的lstm，本文就以pytorch自身的LSTM为例，另外补充一些基于它的一些repo。这部分主要参考了：[SEQUENCE MODELS AND LONG-SHORT TERM MEMORY NETWORKS](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html?highlight=lstm)。

序列建模中，不同时间之间的输入是有依赖关系的。经典的序列建模的例子就是隐马尔可夫模型，还有条件随机场。循环神经网络是一种能保存一些状态state的神经网络。它的输出能被用作下一个输入的一部分，因此信息能够随着网络沿序列传播下去。在LSTM中，序列中的每个元素都有对应的隐含状态，它能序列中它之前的点的信息。我们可以用隐含状态来帮助预测。

pytorch的LSTM需要所有的输入是三维张量。张量的每个轴的含义是十分重要的，不可搞错。pytorch lstm是sequence first的。即第一维是序列本身，第二维是一个batch中的东西，第三是输入向量的维度。再具体点说，是这样的：

pytorch 的LSTM单元接收的输入必须是三维张量：

- 第一维反映序列seq的结构，即seq内的个数
- 第二维是minibatch，即一次喂给网络的seq数目
- 第三维是输入元素，每个元素具体是多少维的向量

第一维和第二维和直观上的维度顺序不太一致，因为 pytorch 对tensor是sequence first排列维度的。

- batch first: $a_1 a_2 a_3 a_4 a_5|b_1 b_2 b_3 b_4 b_5$
- sequence first: $a_1 b_1| a_2 b_2| a_3 b_3| a_4 b_4| a_5 b_5$

所以pytorch中不同seq同一时刻对应的输入单元在内存中毗邻，这样可以快速读取数据。

现在看一个例子。

In [1]:
# Author: Robert Guthrie

import torch
import torch.nn as nn # 神经网络模块
import torch.nn.functional as F # 神经网络模块中的常用功能
import torch.optim as optim   # 模型优化器模块

torch.manual_seed(1)

In [2]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>)
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward>))


Pytorch中，LSTM类继承自RNNBase类，RNNBase类继承自Module类，Module类是pytorch中完成一定网络功能的基类，可以通过继承该类定义自己的神经网络。自己实现神经网络时，一般要重写其forward方法。

Module实现了__call__方法，这意味着其可被当做可调用方法使用。比如上面就有直接用到lstm()。

接下来简单分析下上面的代码。

首先，固定随机种子，这样后面的结果能维持不变，这也是实现reproducible结果的必要手段。

然后，定义LSTM网络，定义的时候使用的是输入和输出的维度，这里这个维度是LSTM网络输入输出的三维张量的最后一维的大小，也就是向量本身的长度。

再然后，生成一个长度为5，每一个元素为1\*3的序列作为输入，即输入张量的维度为(5,1,3)。

接下来设置隐藏层维度，初始化隐藏层的数据。hidden变量是一个**元组**，其第一个元素是LSTM**隐藏层输出**，另一个元素维护**隐藏层的状态**。每个元素都是torch.rand(1,1,3)，就是一个维度为(1,1,3)的以一定高斯分布生成的张量。三维的含义和输入的是一致的。

接下来的前向计算有两种方式。

一是沿序列一次计算一步，每步计算之后，hidden变量都会包含hidden state。

另一种是一次计算整个序列，cat函数可以参考：https://pytorch.org/docs/stable/torch.html?highlight=cat#torch.cat ，简而言之就是拼接功能，将输入拼接起来，并转为一个三维(len(inputs), 1, -1)的张量。

## dropout

参考：[Dropout在RNN中的应用综述](https://lonepatient.top/2018/09/24/a-review-of-dropout-as-applied-to-rnns.html)，及原文[A review of Dropout as applied to RNNs](https://medium.com/@bingobee01/a-review-of-dropout-as-applied-to-rnns-72e79ecd5b7b)

前面已经说过了Dropout基本概念。

RNN与仅前馈神经网络的不同之处在于**先前的状态反馈到网络中，允许网络保留先前状态**。因此，将标准dropout应用于RNN会**限制网络保留先前状态的能力**，从而阻碍其性能。 Bayer等人指出了将dropout应用于递归神经网络（RNN）的问题。如果将完整的输出权重向量设置为零，则“在每次前向传递期间导致RNN动态变化是非常显著的。”

首先看下RNN的前向计算代码：

```python
class RNN:
# ...
    def step(self, x):
        # update the hidden state
        self.h = np.tanh(np.dot(self.W_hh, self.h) + np.dot(self.W_xh, x))
        # compute the output vector
        y = np.dot(self.W_hy, self.h)
        return y
rnn = RNN()
y = rnn.step(x) # x is an input ve
```

作为克服应用于RNN的dropout性能问题的一种方法，Zaremba和Pham等仅将dropout应用于非循环连接（Dropout未应用于隐藏状态）。 “通过不对循环连接使用dropout，LSTM可以从dropout正则化中受益，而不会牺牲其宝贵的记忆能力。”

![](img/79805875.jpg)

即 Dropout仅适用于非循环连接（即仅应用于前馈虚线）。 粗线显示了LSTM中典型的信息流路径。 信息受到dropoutL + 1次的影响，其中L是网络的深度。

Gal和Ghahramani（2015）分析了将Dropout应用于仅前馈RNN的部分，发现这种方法仍然导致过拟合。 他们提出了“**变分Dropout**”，通过**重复“输入，输出和循环层的每个时间步长相同的dropout掩码（在每个时间步骤丢弃相同的网络单元）**”，使用贝叶斯解释，他们看到了语言的改进 建模和情感分析任务超过’纯dropout’。

![](img/14161715.jpg)

即朴素dropout（a）在不同的时间步长使用不同的掩模，在循环层上没有dropout。 变分dropout（b）在每个时间步长使用相同的压差掩模，包括循环层（颜色表示Dropout掩模，实线表示dropout，虚线表示没有dropout的标准连接）。

与Moon和Gal和Ghahramani一样，Semeniuta等人提出**将dropout应用于RNN的循环连接**，以便可以**对循环权重进行正则化以提高性能**。 Gal和Ghahramani使用**网络的隐藏状态**作为计算门值和小区更新以及使用dropout的子网络的输入来正则化子网络（上图b）。 Semeniuta等人的不同之处在于他们认为“整个架构以隐藏状态为关键部分并使整个网络正则化”。 这类似于Moon等人的概念（如上图a所示），但Semeniuta等人发现根据Moon等人直接丢弃先前的状态产生了混合结果，而将dropout应用于隐藏状态更新向量是一种更有原则的方法。

![](img/52714967.jpg)

上图是 Semeniuta等人论文中在LSTM网络的循环连接中三种类型的dropout的例证。 虚线箭头表示断开的连接。 为清楚起见，省略了输入连接。“ 注意Semeniuta等人如何。 （2016）将重复dropout应用于LSTM存储器单元的更新。

设置权重dropout的代码关键部分：

In [3]:
def _setweights(self):
    for name_w in self.weights:
        raw_w = getattr(self.module, name_w + '_raw')
        w = None
        if self.variational:
            mask = torch.autograd.Variable(torch.ones(raw_w.size(0), 1))
            if raw_w.is_cuda: mask = mask.cuda()
            mask = torch.nn.functional.dropout(mask, p=self.dropout,
             training=True)
            w = mask.expand_as(raw_w) * raw_w
        else:
            w = torch.nn.functional.dropout(raw_w, p=self.dropout,
         training=self.training)
        setattr(self.module, name_w, w)

可以参考一个实例进一步了解，这里就不赘述了--博客 [How to Develop Multi-Step LSTM Time Series Forecasting Models for Power Usage](https://machinelearningmastery.com/how-to-develop-lstm-models-for-multi-step-time-series-forecasting-of-household-power-consumption/)

## stacked lstm

先对stacked lstm有个基本认识，参考：[Single Layer & Multi-layer Long Short-Term Memory (LSTM) Model with Intermediate Variables for Weather Forecasting](https://www.sciencedirect.com/science/article/pii/S187705091831439X)，一个典型的stacked lstm结构是这样的：

![](img/Picture9.png)

接下来参考[Stacked Long Short-Term Memory Networks](https://machinelearningmastery.com/stacked-long-short-term-memory-networks/)记录一些基本内容。

原始的lstm就是由一个LSTM隐含层和一个标准前向输出层组成。 Stacked LSTM 是对该模型的扩充，它有多个LSTM隐含层，每层有多个记忆单元。

### 为什么要增加深度

神经网络的深度通常被认为是该方法在一系列具有挑战性的预测问题上取得成功的原因。更多的隐含层能让神经网络更深。更深的隐含层目前被认为能从之前的层中重新组合学习到的特征并创建新的更高层次的抽象特征。比如从线到面到体。

一个足够深的神经网络结构能用来估计大部分函数。增加网络的深度提供了一种需要更少的神经元和更快的训练的解决方案。因此，增加深度是一种典型的优化方式。

### Stacked LSTM Architecture

因为LSTM是作用于序列数据，这意味着随着时间推移，层数的增加增加了输入观察的抽象级别。实际上就是将观察结果随时间分块或在不同的时间尺度上表示问题。

引用一段话：

" ... building a deep RNN by stacking multiple recurrent hidden states on top of each other. This approach potentially allows the hidden state at each level to operate at different timescale"

— [How to Construct Deep Recurrent Neural Networks, 2013](https://arxiv.org/abs/1312.6026)

在一些研究中，LSTM的深度比一个给定层的memory cells的个数对模型来说更重要。Stacked LSTMs 现在已经是序列预测问题的一种较稳定的技术。一个Stacked LSTM模型可以由一个LSTM模型和多个LSTM层构成。如下图所示，上面的LSTM层不输出一个值，而是输出一个序列到下一层LSTM。具体来说，每个输入时间步一个输出，而不是所有输入时间步一个输出。